In [4]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Paraíba - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [7]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_PB.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Paraíba - IDH Renda,Paraíba - IDH,Paraíba - IDH Educacao,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Produção de Cimento (t),Paraíba - Desemprego,Paraíba - IDH Longevidade,Paraíba - Consumo de Cimento (t)
0,2014,0.650589,0.687644,0.639727,3.559137e+07,1.740643e+06,7.503408,2.852940e+07,5.676413e+08,2.779340e+09,...,948.952159,1092.104700,22.396949,19.115911,83.444066,4.091372,2266.039323,10.857997,0.781047,1240.232857
1,2015,0.662663,0.701563,0.655360,3.588415e+07,1.726042e+06,7.497228,2.851564e+07,5.179392e+08,2.561731e+09,...,949.008244,1112.063713,22.423455,19.017059,82.914233,4.121849,2276.134358,11.087923,0.795617,1076.248286
2,2016,0.661269,0.701432,0.656392,3.605286e+07,1.712480e+06,7.491294,2.849183e+07,4.524144e+08,2.516859e+09,...,948.977114,1153.430368,22.490255,18.907433,81.942887,4.149759,2291.680633,11.207329,0.795492,928.236857
3,2017,0.661112,0.701199,0.656337,3.609231e+07,1.709809e+06,7.490431,2.847638e+07,4.213689e+08,2.497710e+09,...,948.990503,1103.061547,22.557983,18.973761,81.889312,4.064685,2353.643222,11.081901,0.794900,786.338000
4,2018,0.659213,0.699763,0.655004,3.597983e+07,1.707996e+06,7.483007,2.842205e+07,4.166580e+08,2.797145e+09,...,948.945703,1028.289848,22.636851,19.105216,82.228711,3.916527,2597.624854,11.111648,0.793800,707.565000
5,2019,0.658051,0.698741,0.654732,3.574054e+07,1.711138e+06,7.470361,2.836375e+07,4.462625e+08,3.484252e+09,...,948.840602,909.705535,22.659927,19.150213,82.347115,3.641942,2980.034771,11.192076,0.792146,769.080000
6,2020,0.656782,0.697445,0.654253,3.530025e+07,1.712857e+06,7.451128,2.826871e+07,5.489412e+08,4.114911e+09,...,948.680394,670.047184,22.668347,19.183622,82.453795,3.316913,3260.773751,11.185263,0.789977,1051.694000
7,2021,0.656126,0.695940,0.652549,3.466369e+07,1.708555e+06,7.437102,2.818228e+07,6.638811e+08,4.421848e+09,...,948.609736,482.739403,22.604381,19.202434,82.825721,3.094794,3425.166126,11.146895,0.787611,1112.087000
8,2022,0.656410,0.695752,0.651053,3.389017e+07,1.703835e+06,7.418674,2.807070e+07,7.010332e+08,4.533216e+09,...,948.625562,426.603873,22.508498,19.157764,83.044908,3.023164,3496.801974,11.108224,0.788394,1064.438000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Paraíba - IDH Renda,Paraíba - IDH,Paraíba - IDH Educacao,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Produção de Cimento (t),Paraíba - Desemprego,Paraíba - IDH Longevidade
0,0.650589,0.687644,0.639727,3.559137e+07,1.740643e+06,7.503408,2.852940e+07,5.676413e+08,2.779340e+09,1.954361e+10,...,13.565055,948.952159,1092.104700,22.396949,19.115911,83.444066,4.091372,2266.039323,10.857997,0.781047
1,0.662663,0.701563,0.655360,3.588415e+07,1.726042e+06,7.497228,2.851564e+07,5.179392e+08,2.561731e+09,1.327596e+10,...,13.574441,949.008244,1112.063713,22.423455,19.017059,82.914233,4.121849,2276.134358,11.087923,0.795617
2,0.661269,0.701432,0.656392,3.605286e+07,1.712480e+06,7.491294,2.849183e+07,4.524144e+08,2.516859e+09,1.011370e+10,...,13.373177,948.977114,1153.430368,22.490255,18.907433,81.942887,4.149759,2291.680633,11.207329,0.795492
3,0.661112,0.701199,0.656337,3.609231e+07,1.709809e+06,7.490431,2.847638e+07,4.213689e+08,2.497710e+09,8.456309e+09,...,13.874302,948.990503,1103.061547,22.557983,18.973761,81.889312,4.064685,2353.643222,11.081901,0.794900
4,0.659213,0.699763,0.655004,3.597983e+07,1.707996e+06,7.483007,2.842205e+07,4.166580e+08,2.797145e+09,7.937760e+09,...,14.562648,948.945703,1028.289848,22.636851,19.105216,82.228711,3.916527,2597.624854,11.111648,0.793800
5,0.658051,0.698741,0.654732,3.574054e+07,1.711138e+06,7.470361,2.836375e+07,4.462625e+08,3.484252e+09,1.091282e+10,...,14.522691,948.840602,909.705535,22.659927,19.150213,82.347115,3.641942,2980.034771,11.192076,0.792146
6,0.656782,0.697445,0.654253,3.530025e+07,1.712857e+06,7.451128,2.826871e+07,5.489412e+08,4.114911e+09,1.625333e+10,...,15.163979,948.680394,670.047184,22.668347,19.183622,82.453795,3.316913,3260.773751,11.185263,0.789977


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    1076.248286
1     928.236857
2     786.338000
3     707.565000
4     769.080000
5    1051.694000
6    1112.087000
7    1064.438000
8            NaN
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Paraíba - IDH Renda,Paraíba - IDH,Paraíba - IDH Educacao,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Produção de Cimento (t),Paraíba - Desemprego,Paraíba - IDH Longevidade
0,-2.072830,-2.188288,-2.223701,-1.692229,1.932862,1.340736,1.099170,1.779974,0.019134,2.003355,...,-0.739142,-0.004196,0.323870,-1.309707,0.824139,1.777995,0.533594,-0.754118,-2.026033,-2.175582
1,0.969339,0.646054,0.410141,-0.034138,0.685477,0.753969,0.858730,0.870373,-0.621387,0.401153,...,-0.719149,1.027676,0.575764,-1.043942,-0.323638,0.819675,0.707181,-0.715042,-0.016512,0.675058
2,0.618051,0.619455,0.584129,0.921319,-0.473069,0.190489,0.442510,-0.328802,-0.753466,-0.407217,...,-1.147863,0.454929,1.097835,-0.374146,-1.596516,-0.937221,0.866152,-0.654865,1.027081,0.650469
3,0.578408,0.571928,0.574823,1.144738,-0.701290,0.108528,0.172471,-0.896969,-0.809829,-0.830898,...,-0.080411,0.701272,0.462152,0.304942,-0.826380,-1.034122,0.381590,-0.415017,-0.069146,0.534717
4,0.099961,0.279434,0.350246,0.507742,-0.856205,-0.596470,-0.777034,-0.983185,0.071542,-0.963455,...,1.385839,-0.122987,-0.481510,1.095740,0.699964,-0.420243,-0.462279,0.529397,0.190844,0.319515
5,-0.192929,0.071417,0.304362,-0.847432,-0.587775,-1.797252,-1.795846,-0.441390,2.094006,-0.202939,...,1.300726,-2.056695,-1.978112,1.327114,1.222430,-0.206083,-2.026238,2.009645,0.893767,-0.004177


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    1076.248286
1     928.236857
2     786.338000
3     707.565000
4     769.080000
5    1051.694000
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Paraíba - IDH Renda,Paraíba - IDH,Paraíba - IDH Educacao,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Produção de Cimento (t),Paraíba - Desemprego,Paraíba - IDH Longevidade
6,-0.465776,-0.177781,0.206347,-1.920273,-0.402642,-1.978266,-1.945162,1.16956,2.035192,0.985175,...,1.73888,-2.16547,-2.165323,1.153,1.273537,-0.012156,-2.023353,1.862242,0.736597,-0.391557


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    1112.087
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4096, activation='tanh', input_shape=(train_input.shape[1],)),      
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3978857316, 4213395386, 2863433271, 3742339014, 952933560, 3604274370, 4199061720, 1558875003, 3676385144, 3561463852]


Step: 0 ___________________________________________


2023-08-19 23:03:57.870459: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 23:03:57.871020: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1749: early stopping
val_loss: 1055.7264404296875
winner_seed: 3978857316


Step: 1 ___________________________________________
Epoch 2909: early stopping
val_loss: 19968.9296875


Step: 2 ___________________________________________
Epoch 1741: early stopping
val_loss: 43646.4375


Step: 3 ___________________________________________
Epoch 1773: early stopping
val_loss: 43646.4375


Step: 4 ___________________________________________
Epoch 1892: early stopping
val_loss: 20701.40234375


Step: 5 ___________________________________________
Epoch 3338: early stopping
val_loss: 838.4884643554688
winner_seed: 3604274370


Step: 6 ___________________________________________
Epoch 2148: early stopping
val_loss: 19968.9296875


Step: 7 ___________________________________________
Epoch 1741: early stopping
val_loss: 44.35768127441406
winner_seed: 1558875003


Step: 8 ___________________________________________
Epoch 1888: early stopping
val_loss: 49741.03125


Step: 9 _____________________

In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 332ms/step - loss: 806083.3125 - val_loss: 1070292.0000
Epoch 2/5000
1/1 [==============================] - 0s 37ms/step - loss: 775385.6875 - val_loss: 1260448.6250
Epoch 3/5000
1/1 [==============================] - 0s 34ms/step - loss: 657643.0625 - val_loss: 1079962.8750
Epoch 4/5000
1/1 [==============================] - 0s 34ms/step - loss: 781169.5000 - val_loss: 875152.2500
Epoch 5/5000
1/1 [==============================] - 0s 36ms/step - loss: 698566.8125 - val_loss: 929339.1875
Epoch 6/5000
1/1 [==============================] - 0s 35ms/step - loss: 724942.0000 - val_loss: 872186.5000
Epoch 7/5000
1/1 [==============================] - 0s 35ms/step - loss: 699688.5625 - val_loss: 795130.8750
Epoch 8/5000
1/1 [==============================] - 0s 34ms/step - loss: 681363.1875 - val_loss: 751511.8125
Epoch 9/5000
1/1 [==============================] - 0s 36ms/step - loss: 665087.2500 - val_loss: 721333.3750
Epoch 10/5000
1

1/1 [==============================] - 0s 36ms/step - loss: 179585.7969 - val_loss: 185545.2969
Epoch 76/5000
1/1 [==============================] - 0s 37ms/step - loss: 176033.3125 - val_loss: 181777.0156
Epoch 77/5000
1/1 [==============================] - 0s 35ms/step - loss: 172551.7969 - val_loss: 178089.0000
Epoch 78/5000
1/1 [==============================] - 0s 36ms/step - loss: 169139.8594 - val_loss: 174479.5781
Epoch 79/5000
1/1 [==============================] - 0s 36ms/step - loss: 165796.0781 - val_loss: 170946.8750
Epoch 80/5000
1/1 [==============================] - 0s 36ms/step - loss: 162519.0625 - val_loss: 167489.1562
Epoch 81/5000
1/1 [==============================] - 0s 35ms/step - loss: 159307.4844 - val_loss: 164104.6562
Epoch 82/5000
1/1 [==============================] - 0s 36ms/step - loss: 156160.0469 - val_loss: 160791.7031
Epoch 83/5000
1/1 [==============================] - 0s 34ms/step - loss: 153075.4375 - val_loss: 157548.6719
Epoch 84/5000
1/1 [=====

Epoch 150/5000
1/1 [==============================] - 0s 37ms/step - loss: 40750.6328 - val_loss: 41512.9688
Epoch 151/5000
1/1 [==============================] - 0s 35ms/step - loss: 39965.2461 - val_loss: 40709.4648
Epoch 152/5000
1/1 [==============================] - 0s 36ms/step - loss: 39195.4766 - val_loss: 39921.9141
Epoch 153/5000
1/1 [==============================] - 0s 37ms/step - loss: 38441.0195 - val_loss: 39149.9258
Epoch 154/5000
1/1 [==============================] - 0s 36ms/step - loss: 37701.5859 - val_loss: 38393.2266
Epoch 155/5000
1/1 [==============================] - 0s 36ms/step - loss: 36976.8633 - val_loss: 37651.4648
Epoch 156/5000
1/1 [==============================] - 0s 35ms/step - loss: 36266.5781 - val_loss: 36924.4219
Epoch 157/5000
1/1 [==============================] - 0s 37ms/step - loss: 35570.4102 - val_loss: 36211.7383
Epoch 158/5000
1/1 [==============================] - 0s 35ms/step - loss: 34888.1172 - val_loss: 35513.1094
Epoch 159/5000
1/1 

1/1 [==============================] - 0s 36ms/step - loss: 10024.1523 - val_loss: 9848.0303
Epoch 226/5000
1/1 [==============================] - 0s 37ms/step - loss: 9850.2295 - val_loss: 9665.0117
Epoch 227/5000
1/1 [==============================] - 0s 36ms/step - loss: 9679.7656 - val_loss: 9485.5049
Epoch 228/5000
1/1 [==============================] - 0s 35ms/step - loss: 9512.6982 - val_loss: 9309.4463
Epoch 229/5000
1/1 [==============================] - 0s 36ms/step - loss: 9348.9541 - val_loss: 9136.7881
Epoch 230/5000
1/1 [==============================] - 0s 35ms/step - loss: 9188.4717 - val_loss: 8967.4678
Epoch 231/5000
1/1 [==============================] - 0s 36ms/step - loss: 9031.1816 - val_loss: 8801.3809
Epoch 232/5000
1/1 [==============================] - 0s 35ms/step - loss: 8877.0146 - val_loss: 8638.5020
Epoch 233/5000
1/1 [==============================] - 0s 39ms/step - loss: 8725.9248 - val_loss: 8478.7412
Epoch 234/5000
1/1 [==============================]

1/1 [==============================] - 0s 36ms/step - loss: 3181.2869 - val_loss: 2477.7717
Epoch 302/5000
1/1 [==============================] - 0s 40ms/step - loss: 3143.5371 - val_loss: 2435.0286
Epoch 303/5000
1/1 [==============================] - 0s 35ms/step - loss: 3106.5420 - val_loss: 2393.0752
Epoch 304/5000
1/1 [==============================] - 0s 36ms/step - loss: 3070.2795 - val_loss: 2351.8948
Epoch 305/5000
1/1 [==============================] - 0s 36ms/step - loss: 3034.7393 - val_loss: 2311.4827
Epoch 306/5000
1/1 [==============================] - 0s 41ms/step - loss: 2999.9050 - val_loss: 2271.8279
Epoch 307/5000
1/1 [==============================] - 0s 42ms/step - loss: 2965.7664 - val_loss: 2232.8970
Epoch 308/5000
1/1 [==============================] - 0s 43ms/step - loss: 2932.3074 - val_loss: 2194.6973
Epoch 309/5000
1/1 [==============================] - 0s 35ms/step - loss: 2899.5137 - val_loss: 2157.1953
Epoch 310/5000
1/1 [==============================] 

...

1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1386/5000
1/1 [==============================] - 0s 39ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1387/5000
1/1 [==============================] - 0s 44ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1388/5000
1/1 [==============================] - 0s 40ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1389/5000
1/1 [==============================] - 0s 40ms/step - loss: 1284.3564 - val_loss: 44.3577
Epoch 1390/5000
1/1 [==============================] - 0s 37ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1391/5000
1/1 [==============================] - 0s 41ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1392/5000
1/1 [==============================] - 0s 37ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1393/5000
1/1 [==============================] - 0s 37ms/step - loss: 1284.3564 - val_loss: 44.3577
Epoch 1394/5000
1/1 [==============================] - 0s 38ms

1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1463/5000
1/1 [==============================] - 0s 46ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1464/5000
1/1 [==============================] - 0s 46ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1465/5000
1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1466/5000
1/1 [==============================] - 0s 40ms/step - loss: 1284.3564 - val_loss: 44.3577
Epoch 1467/5000
1/1 [==============================] - 0s 37ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1468/5000
1/1 [==============================] - 0s 37ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1469/5000
1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1470/5000
1/1 [==============================] - 0s 40ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1471/5000
1/1 [==============================] - 0s 37ms

1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1540/5000
1/1 [==============================] - 0s 39ms/step - loss: 1284.3567 - val_loss: 44.3577
Epoch 1541/5000
1/1 [==============================] - 0s 40ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1542/5000
1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1543/5000
1/1 [==============================] - 0s 43ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1544/5000
1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1545/5000
1/1 [==============================] - 0s 39ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1546/5000
1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1547/5000
1/1 [==============================] - 0s 39ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1548/5000
1/1 [==============================] - 0s 39ms

1/1 [==============================] - 0s 44ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1617/5000
1/1 [==============================] - 0s 43ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1618/5000
1/1 [==============================] - 0s 39ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1619/5000
1/1 [==============================] - 0s 38ms/step - loss: 1284.3564 - val_loss: 44.3577
Epoch 1620/5000
1/1 [==============================] - 0s 35ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1621/5000
1/1 [==============================] - 0s 37ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1622/5000
1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1623/5000
1/1 [==============================] - 0s 38ms/step - loss: 1284.3564 - val_loss: 44.3577
Epoch 1624/5000
1/1 [==============================] - 0s 37ms/step - loss: 1284.3564 - val_loss: 44.3577
Epoch 1625/5000
1/1 [==============================] - 0s 38ms

1/1 [==============================] - 0s 39ms/step - loss: 1284.3564 - val_loss: 44.3577
Epoch 1694/5000
1/1 [==============================] - 0s 39ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1695/5000
1/1 [==============================] - 0s 37ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1696/5000
1/1 [==============================] - 0s 36ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1697/5000
1/1 [==============================] - 0s 37ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1698/5000
1/1 [==============================] - 0s 38ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1699/5000
1/1 [==============================] - 0s 39ms/step - loss: 1284.3564 - val_loss: 44.3577
Epoch 1700/5000
1/1 [==============================] - 0s 40ms/step - loss: 1284.3566 - val_loss: 44.3577
Epoch 1701/5000
1/1 [==============================] - 0s 40ms/step - loss: 1284.3567 - val_loss: 44.3577
Epoch 1702/5000
1/1 [==============================] - 0s 40ms

In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[1112.087](test_target) - [[1045.0338]](prediction) = 67.05318652343749


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Paraíba - IDH Renda,Paraíba - IDH,Paraíba - IDH Educacao,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Produção de Cimento (t),Paraíba - Desemprego,Paraíba - IDH Longevidade
0,-1.406316,-1.414166,-1.412050,-1.318709,1.239538,1.232926,1.102373,1.165029,1.396038,1.335266,...,0.655901,-1.177415,-1.060901,-1.017294,1.203111,1.089085,-1.242287,-1.129477,-1.331868,-1.414174
1,0.832414,0.717083,0.638303,0.216912,-0.030143,-0.016529,0.216003,0.111769,-0.502296,-0.264151,...,0.757105,1.267140,-0.279406,-0.342138,0.042178,0.236752,0.035871,-0.172291,0.254114,0.716275
2,0.573902,0.697083,0.773747,1.101797,-1.209395,-1.216397,-1.318376,-1.276798,-0.893743,-1.071116,...,-1.413006,-0.089726,1.340307,1.359432,-1.245289,-1.325837,1.206415,1.301767,1.077754,0.697898


train_target:


,Paraíba - Consumo de Cimento (t)
0,1076.248286
1,928.236857
2,786.338000


test_input:


...

,Paraíba - IDH Renda,Paraíba - IDH,Paraíba - IDH Educacao,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - Produção de Cimento (t),Paraíba - Desemprego,Paraíba - IDH Longevidade
6,-0.465776,-0.177781,0.206347,-1.920273,-0.402642,-1.978266,-1.945162,1.16956,2.035192,0.985175,...,1.73888,-2.16547,-2.165323,1.153,1.273537,-0.012156,-2.023353,1.862242,0.736597,-0.391557


test_target:


,Paraíba - Consumo de Cimento (t)
6,1112.087


Epoch 1741: early stopping
1/1 [==============================] - 0s 33ms/step
Error: 67.05318652343749




[786.3334350585938, 746.9500732421875, 738.1480712890625, 1045.0338134765625]

In [21]:
display(targets)
display(predictions)

[707.565, 769.08, 1051.694, 1112.087]

[786.3334350585938, 746.9500732421875, 738.1480712890625, 1045.0338134765625]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

120.3743692626953

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.13226404740840253